<a href="https://colab.research.google.com/github/RedBatProject/advanced-works/blob/main/mask_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="sbunlp/fabert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
text="""
زندگی هر روز [MASK] جدیدی برای یاد گیری و پیشرفت به ما میدهد
"""

In [ ]:
pipe(text,top_k=10)

[{'score': 0.2842203676700592,
  'token': 11036,
  'token_str': 'فرصت\u200cهای',
  'sequence': 'زندگی هر روز فرصت\u200cهای جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.18552444875240326,
  'token': 8959,
  'token_str': 'چیزهای',
  'sequence': 'زندگی هر روز چیزهای جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.15934379398822784,
  'token': 4392,
  'token_str': 'فرصت',
  'sequence': 'زندگی هر روز فرصت جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.07509810477495193,
  'token': 6385,
  'token_str': 'راه\u200cهای',
  'sequence': 'زندگی هر روز راه\u200cهای جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.05382407084107399,
  'token': 13265,
  'token_str': 'درس\u200cهای',
  'sequence': 'زندگی هر روز درس\u200cهای جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.04413863644003868,
  'token': 2703,
  'token_str': 'چیز',
  'sequence': 'زندگی هر روز چیز جدیدی برای یاد گیری و پیشرفت به ما میدهد'},
 {'score': 0.022273559123277664,
  'token': 3233,
  '

In [ ]:
reference = """
زندگی هر روز فرصت‌های جدیدی برای یادگیری و پیشرفت به ما می‌دهد. وقتی به دنیای پیرامونمان نگاه می‌کنیم، می‌توانیم از تجربیات خود و دیگران درس بگیریم. گاهی اوقات چالش‌ها و مشکلاتی که با آنها مواجه می‌شویم، می‌توانند به ما نشان دهند که چقدر قوی و مقاوم هستیم. در این مسیر، دوستان و خانواده نقش بسیار مهمی دارند. آنها می‌توانند با حمایت و تشویق‌هایشان ما را به جلو هدایت کنند.

یکی از مهم‌ترین نکات در زندگی این است که همیشه با خودمان صادق باشیم. وقتی با خودمان صادق باشیم، می‌توانیم بهتر تصمیم بگیریم و به اهدافمان نزدیک‌تر شویم. همچنین باید یاد بگیریم که از اشتباهاتمان درس بگیریم. هر اشتباه یک فرصت برای یادگیری است و می‌تواند به ما کمک کند تا در آینده بهتر عمل کنیم.

خواندن کتاب، سفر کردن و گفتگو با افراد مختلف می‌تواند دید ما را نسبت به جهان گسترش دهد. هر کدام از این فعالیت‌ها می‌تواند به ما کمک کند تا چیزهای جدیدی یاد بگیریم و درک بهتری از زندگی پیدا کنیم. در نهایت، مهم‌ترین چیز این است که همیشه با امید و انگیزه به زندگی نگاه کنیم و هرگز دست از تلاش برنداریم.
"""

wrong = """
زندگی هر روز فرست‌های جدیدی برای یاد گیری و پیشرفت به ما میدهد. وقتی به دنیای پیرامونمان نگاه میکنیم، میتوانیم از تجریبات خود و دیگران درس بگیریم. گاهی اوقات چالش‌ها و مشکلاتی که با آنها مواجه میشویم، میتوانند به ما نشان دهند که چقدر قوی و مقاوم هستیم. در این مسیر، دوستان و خانوده نقش بسیار مهمی دارند. انها میتوانند با حمایت و تشویق‌هایشان ما را به جلو هدایت کنند.

یکی از مهمترین نکات در زندگی این است که همیشه با خودمان صادق باشیم. وقتی با خودمان صادق باشیم، میتوانیم بهتر تصمیم بگیریم و به اهدافمان نزدیک‌تر شویم. همچنین باید یاد بگیریم که از اشتباهاتمان درس بگیریم. هر اشتباه یک فرصت برای یاد گیری است و میتواند به ما کمک کند تا در اینده بهتر عمل کنیم.

خواندن کتاب، سفر کردن و گفتگو با افراد مختلف میتواند دید ما را نسبت به جهان گسترش دهد. هر کدام از این فعالیت‌ها میتواند به ما کمک کند تا چیزهای جدیدی یاد بگیریم و درک بهتری از زندگی پیدا کنیم. در نهایت، مهمترین چیز این است که همیشه با امید و انگیزه به زندگی نگاه کنیم و هر گز دست از تلاش برنداریم.
"""



In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("sbunlp/fabert"')
# model = AutoModelForMaskedLM.from_pretrained("sbunlp/fabert")'

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained("sbunlp/fabert")
# model = AutoModelForMaskedLM.from_pretrained("sbunlp/fabert")
# Tokenize input
sentence = "This is a sample sentence."
inputs = tokenizer(sentence, return_tensors='pt')

# Get hidden states from the model
with torch.no_grad():
    outputs = model(**inputs)

# The last hidden state of the [CLS] token
cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()

print(cls_embedding)


tensor([-1.9931e-01, -2.1006e-01, -1.9499e-01, -3.3238e-01, -5.2126e-01,
        -3.7337e-01,  1.5419e-01,  4.0006e-01,  3.5362e-02, -1.5012e-01,
        -3.5267e-01, -2.2861e-01, -9.4948e-02,  1.0328e-01,  5.1352e-01,
        -9.0901e-02,  2.3864e-01,  4.8223e-01,  3.1379e-01, -4.0566e-01,
         9.8627e-02, -8.8768e-02, -2.7461e-01, -5.0871e-01,  2.0913e-01,
        -3.0627e-01, -4.2953e-02, -4.1310e-01, -4.0709e-02,  8.0826e-02,
        -6.0292e-02,  3.5720e-01, -3.4175e-01, -1.4824e-01,  3.9080e-01,
        -1.3152e-01,  3.6742e-01, -9.1257e-03,  4.3404e-01,  1.4182e-01,
        -2.6785e-01, -8.7464e-02,  2.6639e-01,  5.4087e-02, -5.5894e-02,
        -5.7430e-01, -2.8187e+00, -3.4312e-01, -2.7937e-01, -3.0941e-01,
        -8.8544e-02, -2.2791e-01,  4.0630e-01,  5.0902e-01, -2.2796e-01,
         3.7731e-01, -2.9018e-01,  3.1223e-01,  1.9454e-01, -1.0551e-01,
         1.5180e-01, -6.6327e-02, -5.1237e-02, -1.4962e-01, -1.7307e-01,
         6.6952e-01, -1.2911e-01,  3.7059e-01, -3.5

In [135]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained model and tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("sbunlp/fabert")
model = AutoModelForMaskedLM.from_pretrained("sbunlp/fabert")
# Set the model to evaluation mode
model.eval()

# Prepare the input sentence
sentence = "سلام بر تو [MASK] من."
inputs = tokenizer(sentence, return_tensors='pt')

# Get predictions for the masked token
with torch.no_grad():
    outputs = model(**inputs)




In [136]:
# Get the index of the masked token
masked_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

# masked_index = 1

In [137]:
# masked_index = torch.tensor(0).unsqueeze(0)
masked_index

tensor([4])

In [138]:
# Get the predicted token IDs for the masked position
predicted_token_ids = torch.topk(outputs.logits[0, masked_index], k=5).indices


In [139]:

# Convert token IDs to tokens
# predicted_tokens = [tokenizer.decode(token_id) for token_id in predicted_token_ids[0]]

predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]


In [140]:
# predicted_tokens = [''.join(tok for tok in tokenizer.decode(token_id).split(' ')) for token_id in predicted_token_ids[0]]


In [141]:
predicted_tokens

['آقای', 'مولای', 'عشق', 'حسین', 'عزیز']